## As discussed in the report we need to discard bad examples. 

In [1]:
import os
import PIL
from PIL import Image
import glob
import shutil

def check_and_discard_data(data_dir , dump_dir):
    for class_name in os.listdir(data_dir):
        list_of_files_of_single_class = glob.glob(os.path.join(data_dir , class_name , '*')  )  
        for file_name in list_of_files_of_single_class:
            if file_name.lower().endswith(( '.jpg', '.jpeg')):
                try:
                    img=Image.open(file_name)
                    img.verify()
                    height , width = img.size
                    if height < 80 or width < 80:
                        print('dim less')
                        shutil.move(file_name, dump_dir)
                except(IOError,SyntaxError)as e:
                    print('io error')
                    shutil.move(file_name, dump_dir)
            else:
                print('print other format')
                shutil.move( file_name , dump_dir)

In [1]:
check_and_discard_data('nsfw_datset' , 'dump')

In [4]:
len(os.listdir('dump'))

16864

## Spliting the datset into train , test , validation

In [2]:
!pygmentize useful_scripts/useful_scripts/split_dataset.py

import os
import numpy as np
import shutil

# # Creating Train / Val / Test folders (One time use)

def split_dataset_one_time(root_dir , valid_size, test_size , train_path , valid_path , test_path):
    

    class_names = os.listdir(root_dir)

    #print(class_names)
    for single_class in class_names:
        try :


            os.makedirs(os.path.join( train_path , single_class))
            os.makedirs(os.path.join( valid_path , single_class))
            os.makedirs(os.path.join( test_path , single_class))


            train_single_path = os.path.join( train_path , single_class)
            test_single_path = os.path.join( test_path , single_class)
            valid_single_path = os.path.join( valid_path , single_class)
        except FileExistsError:
            pass
        
        single_full_list = os.listdir(os.path.join(root_dir , single_class))
        #print(single_full_list[0])

        num_examples_per_class = len(single_full_list)
        indices = list(range(num_e

In [2]:
from useful_scripts.useful_scripts.split_dataset import split_dataset_one_time

In [3]:
split_dataset_one_time('nsfw_datset' , 0.075 ,0.075 , 'nsfw_dataset_train' , 'nsfw_dataset_valid' , 'nsfw_dataset_test')

## Setting the im2rec tool in the machine. 

In [1]:
import sys,os

suffix='/mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
%env IM2REC=$im2rec

env: IM2REC=/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/mxnet/tools/im2rec.py


## We need to create .lst file before creating .rec file Notice That test_ratio is kept at 0.0 because we have already split the dataset. Also The --resize is set 224 because image classification algorithm accepts image size (224,224).

In [2]:
%%bash
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.0 --train-ratio=1.0 nsfw_dataset_valid nsfw_dataset_valid > nsfw_dataset_valid_classes

echo "Label classes:"
cat nsfw_dataset_valid_classes

echo "Creating RecordIO files"
python $IM2REC --num-thread=4 nsfw_dataset_valid.lst nsfw_dataset_valid/ --resize=224
#python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
animated 0
nude 1
porn 2
safe_for_work 3
semi_nude 4
Creating RecordIO files
Creating .rec file from /home/ec2-user/SageMaker/nsfw_dataset_valid.lst in /home/ec2-user/SageMaker
time: 0.3074500560760498  count: 0
time: 28.615877151489258  count: 1000
time: 29.252779245376587  count: 2000
time: 27.263898849487305  count: 3000
time: 24.09006953239441  count: 4000
time: 24.43202519416809  count: 5000
time: 23.392404556274414  count: 6000
time: 22.804959058761597  count: 7000
time: 23.166723251342773  count: 8000
time: 21.34347176551819  count: 9000
time: 20.13561248779297  count: 10000
time: 20.357985973358154  count: 11000
-rw-rw-r-- 1 ec2-user ec2-user 387M Sep 11 18:49 nsfw_dataset_test.rec
-rw-rw-r-- 1 ec2-user ec2-user 318M Sep 12 07:16 nsfw_dataset_valid.rec


## upload the .rec file to s3.

In [3]:
!aws s3 cp nsfw_dataset_valid.rec s3://project-completion-udacity/nsfw_dataset/validation/

upload: ./nsfw_dataset_valid.rec to s3://project-completion-udacity/nsfw_dataset/validation/nsfw_dataset_valid.rec


## We did for validation. Now doing for testing.

In [8]:
%%bash
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.0 --train-ratio=1.0 nsfw_dataset_test nsfw_dataset_test > nsfw_dataset_test_classes

echo "Label classes:"
cat nsfw_dataset_test_classes

echo "Creating RecordIO files"
python $IM2REC --num-thread=4 nsfw_dataset_test.lst nsfw_dataset_test/ --resize=256
#python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
animated 0
nude 1
porn 2
safe_for_work 3
semi_nude 4
Creating RecordIO files
Creating .rec file from /home/ec2-user/SageMaker/nsfw_dataset_test.lst in /home/ec2-user/SageMaker
time: 0.05958127975463867  count: 0
time: 9.245877742767334  count: 1000
time: 7.712120532989502  count: 2000
time: 7.699880361557007  count: 3000
time: 9.303281545639038  count: 4000
time: 9.592679262161255  count: 5000
time: 8.724056959152222  count: 6000
time: 8.638715028762817  count: 7000
time: 10.148676872253418  count: 8000
time: 7.879869699478149  count: 9000
time: 7.991632699966431  count: 10000
time: 8.284252405166626  count: 11000
-rw-rw-r-- 1 ec2-user ec2-user 387M Sep 11 18:49 nsfw_dataset_test.rec
-rw-rw-r-- 1 ec2-user ec2-user 388M Sep 11 18:46 nsfw_dataset_valid.rec


In [9]:
!aws s3 cp nsfw_dataset_test.rec s3://project-completion-udacity/nsfw_dataset/testing/

upload: ./nsfw_dataset_test.rec to s3://project-completion-udacity/nsfw_dataset/testing/nsfw_dataset_test.rec


## Similarly doing this training .

In [4]:
%%bash
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.0 --train-ratio=1.0 nsfw_dataset_train nsfw_dataset_train > nsfw_dataset_train_classes

echo "Label classes:"
cat nsfw_dataset_train_classes

echo "Creating RecordIO files"
python $IM2REC --num-thread=4 nsfw_dataset_train.lst nsfw_dataset_train/ --resize=224
#python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
animated 0
nude 1
porn 2
safe_for_work 3
semi_nude 4
Creating RecordIO files
Creating .rec file from /home/ec2-user/SageMaker/nsfw_dataset_train.lst in /home/ec2-user/SageMaker
imread read blank (None) image for file: /home/ec2-user/SageMaker/nsfw_dataset_train/porn/le8LkPs.jpg
time: 0.046601057052612305  count: 0
time: 18.696394205093384  count: 1000
time: 17.1350200176239  count: 2000
time: 17.429579734802246  count: 3000
time: 15.988012313842773  count: 4000
time: 15.514463901519775  count: 5000
time: 16.88562774658203  count: 6000
time: 14.923904657363892  count: 7000
time: 15.456107378005981  count: 8000
time: 14.62105655670166  count: 9000
time: 15.861175060272217  count: 10000
time: 12.652273893356323  count: 11000
time: 12.256561517715454  count: 12000
time: 15.415345668792725  count: 13000
time: 12.36422848701477  count: 14000
time: 13.456058979034424  count: 15000
time: 12.073819398880005  count: 16000
time: 12.335830688476562  count: 17000
t

In [5]:
!aws s3 cp nsfw_dataset_train.rec s3://project-completion-udacity/nsfw_dataset/training/

upload: ./nsfw_dataset_train.rec to s3://project-completion-udacity/nsfw_dataset/training/nsfw_dataset_train.rec
